In [1]:
%%init_spark
launcher.conf.spark.app.name = "avinash"
launcher.conf.spark.local.dir = "/data/tmp/spark"
launcher.conf.spark.driver.memory = "40g"
launcher.conf.spark.executor.memory = "50g"
launcher.conf.spark.executors = "21"
launcher.conf.spark.executor.cores = "21"
launcher.conf.spark.sql.shuffle.partitions = 210
launcher.conf.spark.sql.shuffle.minPartitions = 20
launcher.conf.spark.ui.showConsoleProgress = "true"
launcher.conf.spark.yarn.keytab = "/home/jioapp/aditya/jiomart_cluster/couture.keytab"
launcher.conf.spark.yarn.principal = "couture@COUTURE.AI"
launcher.conf.spark.driver.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"
launcher.conf.spark.executor.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"

In [2]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.166.181.219:4043
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1750414972267)
SparkSession available as 'spark'


import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [3]:
def readP(path:String)={
    var df = spark.read.parquet(raw"hdfs://10.144.96.170:8020/$path")
    df
}

readP: (path: String)org.apache.spark.sql.DataFrame


In [4]:
var categoryAffinityDF=readP("/data1/archive/avinash/RECO/userFeaturesFinal")

categoryAffinityDF: org.apache.spark.sql.DataFrame = [user_id: string, avg_bag_value: double ... 18 more fields]


In [5]:
var explodeAffinity = categoryAffinityDF
.withColumn("exploded_affinity",explode(col("attribute_affinity_brandname")))
.withColumn("brandname",col("exploded_affinity.attribute"))
.withColumn("score",col("exploded_affinity.affinity_score"))
.select("user_id","brandname","score")

explodeAffinity: org.apache.spark.sql.DataFrame = [user_id: string, brandname: string ... 1 more field]


In [6]:
explodeAffinity.printSchema

root
 |-- user_id: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- score: double (nullable = true)



In [12]:
explodeAffinity = explodeAffinity.limit(1000)

explodeAffinity: org.apache.spark.sql.DataFrame = [user_id: string, brandname: string ... 1 more field]


In [13]:
import org.apache.spark.ml.feature.StringIndexer

import org.apache.spark.ml.feature.StringIndexer


In [14]:
val userIndexer = new StringIndexer()
  .setInputCol("user_id")
  .setOutputCol("userIndex")
  .fit(explodeAffinity)

val brandIndexer = new StringIndexer()
  .setInputCol("brandname")
  .setOutputCol("brandIndex")
  .fit(explodeAffinity)

val indexedDF = userIndexer.transform(explodeAffinity)
  .transform(brandIndexer.transform(_))
  .select(
    col("userIndex").cast("int").alias("user"),
    col("brandIndex").cast("int").alias("item"),
    col("score").cast("float").alias("rating")
  )

userIndexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_11fe52f2d440, handleInvalid=error
brandIndexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_fde77ae8ce9b, handleInvalid=error
indexedDF: org.apache.spark.sql.DataFrame = [user: int, item: int ... 1 more field]


In [15]:
import org.apache.spark.ml.recommendation.ALS

val als = new ALS()
  .setUserCol("user")
  .setItemCol("item")
  .setRatingCol("rating")
  .setRank(3)
  .setMaxIter(10)
  .setRegParam(0.1)
  .setColdStartStrategy("drop")
  .setImplicitPrefs(false)

val model = als.fit(indexedDF)

25/06/20 16:04:20 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/06/20 16:04:20 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.
25/06/20 16:04:20 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/06/20 16:04:20 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.
25/06/20 16:04:20 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerb

import org.apache.spark.ml.recommendation.ALS
als: org.apache.spark.ml.recommendation.ALS = als_a2b1a129bcfb
model: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_a2b1a129bcfb, rank=3


In [16]:
// Example: DataFrame with columns "user" and "item"
val userItemPairs = spark.createDataFrame(Seq(
  (0, 1),
  (0, 5),
  (10, 2)
)).toDF("user", "item")

// Predict ratings for these pairs
val predictions = model.transform(userItemPairs)

predictions.show()

25/06/20 16:04:22 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/06/20 16:04:22 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/06/20 16:04:22 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.
25/06/20 16:04:22 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/06/20 16:04:22 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.k

userItemPairs: org.apache.spark.sql.DataFrame = [user: int, item: int]
predictions: org.apache.spark.sql.DataFrame = [user: int, item: int ... 1 more field]
